<a href="https://colab.research.google.com/github/GabHoo/Challenging-SRL/blob/main/Test_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETTING UP
Run the following cells in the Settin Up section to be able to run any of the tests in this notebook with the two suggested models.  
Change the current value for both model path according to your own models location. If No models are found they will be downloaded

In [6]:
# INSTALL AND IMPORTS
"""
pip install allennlp
pip install allennlp-models
pip install -U spaCy
pip install checklist
"""
from allennlp.predictors import Predictor
import allennlp_models.tagging

import json
import os
from utils import *
import utils
import re


In [7]:
#LOADING MODEL
model="Bert"

if model=="Bert":
    model_name="structured-prediction-srl-bert.2020.12.15.tar.gz"
    path_model="models/"+model_name
elif model=="Bilstm":
    model_name="openie-model.2020.03.26.tar.gz"
    path_model="models/"+model_name
else:
    print("Model not found!")
    exit()

if os.path.exists(path_model):
    print("Model found!")
    predictor = Predictor.from_path(path_model)
else:
    predictor = Predictor.from_path("https://storage.googleapis.com/"+model_name)

#TESTING IF THE MODELS ARE LOADED CORRECTLY

pred=predictor.predict("SRL model was loaded succeffully!")
if pred :
    print((" ").join(pred['words']))
    

Model found!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/gabhoo/.local/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may 

SRL model was loaded succeffully !


# Useful functions


In [8]:
def append_to_results(results_path,your_result):
    """Add results to the score board. 
    your_Results neets to be a dictionary alraedy'
    """
    if type(your_result) != dict:
        print("your_result needs to be a dictionary!")
        return
    
    with open(results_path, 'r') as f:
        data = json.load(f)
    
    data.update(your_result)

    with open(results_path, 'w') as f:
        json.dump(data, f)


# SANDBOX

In [165]:
#copula
pred=predictor_Bert.predict("This is tasty")
pred

{'verbs': [{'verb': 'is',
   'description': '[ARG1: This] [V: is] [ARG2: tasty]',
   'tags': ['B-ARG1', 'B-V', 'B-ARG2']}],
 'words': ['This', 'is', 'tasty']}

In [166]:
#esistential
pred=predictor_Bert.predict("There was sound there")
pred

{'verbs': [{'verb': 'was',
   'description': 'There [V: was] [ARG1: sound] [ARGM-LOC: there]',
   'tags': ['O', 'B-V', 'B-ARG1', 'B-ARGM-LOC']}],
 'words': ['There', 'was', 'sound', 'there']}

In [96]:
#auxiliary
pred=predictor_Bert.predict("I am cooking")
pred

{'verbs': [{'verb': 'am',
   'description': 'I [V: am] cooking',
   'tags': ['O', 'B-V', 'O']},
  {'verb': 'cooking',
   'description': '[ARG0: I] am [V: cooking]',
   'tags': ['B-ARG0', 'O', 'B-V']}],
 'words': ['I', 'am', 'cooking']}

In [99]:
pred=predictor_Bert.predict("There is a cat")
pred

{'verbs': [{'verb': 'is',
   'description': 'There [V: is] [ARG1: a cat]',
   'tags': ['O', 'B-V', 'B-ARG1', 'I-ARG1']}],
 'words': ['There', 'is', 'a', 'cat']}

In [ ]:
pred=predictor_Bert.predict("Brand new test cook Kim sent over a report on her cooking of the shrimp dish.")
pred
#for verb in pred['verbs']:
    #print(verb['verb'],verb['description'],"\n")

In [85]:
pred=predictor_Bert.predict("and I had to like clean the house and do the cooking and watch Sam")
pred
for verb in pred['verbs']:
    print(verb['verb'],"===",verb['description'],"\n")

had === and I [V: had] to like clean the house and do the cooking and watch Sam 

clean === and [ARG0: I] had to [ARGM-DIS: like] [V: clean] [ARG1: the house] and do the cooking and watch Sam 

do === and [ARG0: I] had to [ARGM-DIS: like] clean the house and [V: do] [ARG1: the cooking] and watch Sam 

watch === and [ARG0: I] had to [ARGM-DIS: like] clean the house and do the cooking and [V: watch] [ARG1: Sam] 



In [86]:
pred=predictor_Bert.predict("Twenty five trucks capable of cooking food were dispatched from other states .")
pred
for verb in pred['verbs']:
    print(verb['verb'],"===",verb['description'],"\n")

cooking === [ARG0: Twenty five trucks] capable of [V: cooking] [ARG1: food] were dispatched from other states . 

were === Twenty five trucks capable of cooking food [V: were] dispatched from other states . 

dispatched === [ARG1: Twenty five trucks capable of cooking food] were [V: dispatched] [ARGM-DIR: from other states] . 



In [76]:
pred=predictor_Bert.predict("Alt!")
pred

{'verbs': [], 'words': ['Alt', '!']}

In [62]:
## nested sentences
What u did was bad 

{'verbs': [{'verb': 'are',
   'description': 'you [V: are] annoying me',
   'tags': ['O', 'B-V', 'O', 'O']},
  {'verb': 'annoying',
   'description': '[ARG0: you] are [V: annoying] [ARG1: me]',
   'tags': ['B-ARG0', 'O', 'B-V', 'B-ARG1']}],
 'words': ['you', 'are', 'annoying', 'me']}

In [33]:
pred=predictor_Bert.predict("I could've tried")
pred

{'verbs': [{'verb': 'could',
   'description': "I [V: could] 've tried",
   'tags': ['O', 'B-V', 'O', 'O']},
  {'verb': "'ve",
   'description': "I could [V: 've] tried",
   'tags': ['O', 'O', 'B-V', 'O']},
  {'verb': 'tried',
   'description': "I [ARGM-MOD: could] 've [V: tried]",
   'tags': ['O', 'B-ARGM-MOD', 'O', 'B-V']}],
 'words': ['I', 'could', "'ve", 'tried']}

In [27]:
data=[["it's a wonderfull day", 'it is a wonderfull day'],
   ["There's some pesto left", 'There is some pesto left'],
   ['They will leave the house', "They'll leave the house"],
   ['That would be creazy', "That'd be creazy"],
   ['we are here', "we're here"],
   ['Mark had not see that coming', "Mark hadn't see that coming"],
   ['She will be a great candidate', "She'll be a great candidate"],
   ['Mary is not a nurse.', "Mary isn't a nurse."],
   ["He's gone already", 'He is gone already'],
   ['I would like some tea', "I'd like some tea"],
   ['I would say something to them', "I'd say something to them"],
   ['I could not eat some food now', "I couldn't eat some food now"],
   ["We've decided to change house", 'We have decided to change house'],
   ['I must not lose my temper', "I mustn't lose my temper"]]

In [30]:
for i in data: 
    i1,i2= i
    print(i)
    print(predictor_Bert.predict(i1)['words'])
    print(predictor_Bert.predict(i2)['verbs'])
    print()


["it's a wonderfull day", 'it is a wonderfull day']
['it', "'s", 'a', 'wonderfull', 'day']
[{'verb': 'is', 'description': '[ARG1: it] [V: is] [ARG2: a wonderfull day]', 'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2']}]

["There's some pesto left", 'There is some pesto left']
['There', "'s", 'some', 'pesto', 'left']
[{'verb': 'is', 'description': 'There [V: is] [ARG1: some pesto left]', 'tags': ['O', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']}, {'verb': 'left', 'description': 'There is [ARG1: some pesto] [V: left]', 'tags': ['O', 'O', 'B-ARG1', 'I-ARG1', 'B-V']}]

['They will leave the house', "They'll leave the house"]
['They', 'will', 'leave', 'the', 'house']
[{'verb': "'ll", 'description': "They [V: 'll] leave the house", 'tags': ['O', 'B-V', 'O', 'O', 'O']}, {'verb': 'leave', 'description': "[ARG0: They] [ARGM-MOD: 'll] [V: leave] [ARG1: the house]", 'tags': ['B-ARG0', 'B-ARGM-MOD', 'B-V', 'B-ARG1', 'I-ARG1']}]

['That would be creazy', "That'd be creazy"]
['That', 'would', 'be', 

## LOOKING DATA (propBank)

In [ ]:
import json
import random

In [ ]:
f=open('data_train.jsonl')

In [ ]:
di=json.load(f)

In [ ]:
di = di["data"]
len(di)

40592

In [ ]:
[(' '.join(i['seq_words']),i['BIO']) for i in di[0:6]]

In [ ]:
prediction=predictor.predict(
    sentence="This killing of a respected cleric will be causing us trouble for years to come "
)
prediction_lstm=predictor_bilstm.predict(
    sentence="This killing of a respected cleric will be causing us trouble for years to come "
)

In [ ]:
prediction

In [ ]:
prediction_lstm


# PREDICATE IDENTIFICATION

In [14]:
#In this folder we have the data for the predicate classidication task. Change the path accordingly if files were moved.
path="./Data/Predicate_Identification/"
results_path=f"./results_{model}_Predicate_Identification.json"
## we initialize also the dictionary that will contain the results
with open(results_path, 'w') as f:
    json.dump({"test":"failure_rate"},f)

## VOCABULARY+POS

In [59]:
from utils import *

### Test for contractions

In [46]:
data=json.load(open(path+"contracted_sentences.json"))

In [62]:
failure_rate=evaluate_PI_Contractions_INV(predict_Bert,data)

NameError: name 'evaluate_PI_Contractions_INV' is not defined

In [55]:
pred=predictor_Bert.predict("tooooooooooooO")
get_identified_verbs(pred)

[]

### Test for irregular inflections

In [63]:
data=json.load(open(path+"inflected_sentences.json"))


In [16]:
data

NameError: name 'data' is not defined

In [65]:
evaluate_

## ROBUTSTNESS

## AMBIGUITY

### Experiment with polysemic verbs [POLISEMIC]

In [5]:
print("Tesiting Polysemic words behaviour (DIR) in Preicate Identification\n")

f = open(path+'polysem_verbs_sentences.json')
data = json.load(f)

print("Bart model:")
rate_Bart=evaluate_verbIdentification_DIR(predictor_Bart,data)
print(f"Failure rate {rate_Bart}\n")

print("Bisltm model:")
rate_bilstm=evaluate_verbIdentification_DIR(predictor_Bilstm,data)
print(f"Failure rate {rate_bilstm}\n")

Tesiting Polysemic words behaviour (DIR) in Preicate Identification



NameError: name 'json' is not defined

'I also had a pedicure and they cut my nails too short one of my big toes looks like it s getting infected .'

### Experiment with verbs being in different roles -ing

In [ ]:
The writing of the section on aboriginal culture - the writing is ON PROPBANK | thw writing on the wall looks fresh


when I was in Cyprus the heatARG1 was too annoying - Annoying is on PROPBANK


In [ ]:
sentences = [    "The writing on the wall was a warning of what was to come.",    "The painting of the sunset was breathtakingly beautiful.",    "The singing of the birds was a welcome sound in the morning.","The singing of the birds was a welcome sound in the morning."    "The reading of the will revealed some surprising information.",    "The dancing of the flames mesmerized the audience.",    "The running of the river was soothing to listen to.",    "The howling of the wind kept me awake at night.",    "The barking of the dog alerted us to the intruder.",    "The playing of the piano filled the room with music.",    "The cleaning of the house was a tedious chore.",    "The mowing of the lawn was a necessary task.",    "The washing of the dishes was a never-ending job.",    "The painting of the house was a weekend project.",    "The cooking of the meal took longer than expected.",    "The building of the bridge was a remarkable feat of engineering.",    "The driving of the car was a peaceful way to clear my mind.",    "The typing of the report was interrupted by a power outage.",    "The repairing of the bike took longer than anticipated.",    "The cutting of the cake was the highlight of the party.",    "The playing of the game was a fun way to spend the afternoon."]


In [ ]:
sentences = [    "I have always enjoyed cooking, it is one of my favourite hobbies.",    "The protests that followed the killing of the priest were widespread.",    "The sound of the river running was soothing to listen to.",    "The information revealed in the reading of the will was surprising.",    "The audience was mesmerized by the dancing flames.",    "The sunset was breathtakingly beautiful in the painting.",    "The birds singing in the morning was a welcome sound.",    "The intruder was alerted by the barking of the dog.",    "The music filled the room with the playing of the piano.",    "The tedious chore of cleaning the house took all day.",    "The necessary task of mowing the lawn was finally finished.",    "The never-ending job of washing the dishes always seemed to pile up.",    "The remarkable feat of engineering was achieved in the building of the bridge.",    "The peaceful way to clear my mind was through driving the car.",    "The power outage interrupted the typing of the report.",    "The longer than anticipated repair of the bike was finally completed.",    "The highlight of the party was cutting the cake.",    "The fun way to spend the afternoon was playing the game.",    "The weekend project of painting the house was finally done.",    "The wind howling kept me awake at night.",]


In [ ]:
{"seq_words": ["In", "Baghdad", "the", "fighting", "still", "continues", "in", "several", "areas", "mostly", "in", "Sadr", "city", "and", "Adhamiya", "."], "BIO": ["O", "ARGM-LOC", "O", "ARG1", "ARGM-TMP", "V", "O", "O", "ARGM-LOC", "O", "O", "O", "ARGM-LOC", "O", "O", "O"], "pred_sense": [5, "continue.01"]},
	{"seq_words": ["In", "Baghdad", "the", "fighting", "still", "continues", "in", "several", "areas", "mostly", "in", "Sadr", "city", "and", "Adhamiya", "."], "BIO": ["O", "O", "O", "V", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"], "pred_sense": [3, "fight.01"]},


    following
    the ruling party
    the highest ranking 
    the running time

In [41]:
"""
can you create 5 example sentence where there is a verb not in the semantic role of the verb such as:
 "cooking is one of my favourite hobbies", or "the killing of the priest sparked a lot of protests" 
"""
#se=[ "The birds singing was a welcome sound in the morning.", "The birds's singing was a welcome sound in the morning.", "The singing of the birds was a welcome sound in the morning." ]

se=["cooking is what I love","I love cooking ","I love cooking in the morning", "I loved cooking with you","I have always liked cooking","cooking is a great hooby","cooking vegetables is a great hobby"]

In [ ]:
for s in se:
  print(s)
  pred=predictor.predict(s)
  
  verbs=[x['verb'] for x in pred['verbs']]
  #print(pred)
  print(verbs)
  #print(pred)

cooking is what I love
['is', 'love']
I love cooking 
['love', 'cooking']
I love cooking in the morning
['love', 'cooking']
I loved cooking with you
['loved', 'cooking']
I have always liked cooking
['have', 'liked', 'cooking']
cooking is a great hooby
['is']
cooking vegetables is a great hobby
['cooking', 'is']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  verbs=[x['verb'] for x in pred['verbs']]
  print(verbs)
  #print(pred)

I have always enjoyed cooking, it is one of my favourite hobbies.
['have', 'enjoyed', 'is']
The protests that followed the killing of the priest were widespread.
['followed', 'were']
The sound of the river running was soothing to listen to.
['running', 'was', 'soothing', 'listen']
The information revealed in the reading of the will was surprising.
['revealed', 'was']
The audience was mesmerized by the dancing flames.
['was', 'mesmerized']
The sunset was breathtakingly beautiful in the painting.
['was']
The birds singing in the morning was a welcome sound.
['singing', 'was']
The intruder was alerted by the barking of the dog.
['was', 'alerted']
The music filled the room with the playing of the piano.
['filled']
The tedious chore of cleaning the house took all day.
['cleaning', 'took']
The necessary task of mowing the lawn was finally finished.
['mowing', 'was', 'finished']
The never-ending job of washing the dishes always seemed to pile up.
['ending', 'washing', 'seemed', 'pile']
The re

In [ ]:
prediction=predictor.predict(
    sentence="I was writing a thesis"
)

In [ ]:
prediction

{'verbs': [{'verb': 'was',
   'description': 'I [V: was] writing a thesis',
   'tags': ['O', 'B-V', 'O', 'O', 'O']},
  {'verb': 'writing',
   'description': '[ARG0: I] was [V: writing] [ARG1: a thesis]',
   'tags': ['B-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1']}],
 'words': ['I', 'was', 'writing', 'a', 'thesis']}

In [ ]:
from nltk.corpus import propbank

[nltk_data] Downloading package propbank to /root/nltk_data...


True

In [ ]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
pb_instances = propbank.instances()

In [ ]:
inst=pb_instances[103]

In [ ]:
 import nltk.corpus.reader.propbank as pb

In [ ]:
turn_01 = propbank.roleset('turn.01')

In [ ]:
for role in turn_01.findall("roles/role"):
  print(role.attrib['n'], role.attrib['descr'])

0 turner
1 thing turning
m direction, location


In [ ]:
sentences=["The birds singing in the morning was a welcome sound.","The singing of birds in the morning was a welcome sound."]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

The birds singing in the morning was a welcome sound.
BERT
['[ARG0: The birds] [V: singing] [ARGM-TMP: in the morning] was a welcome sound .', '[ARG1: The birds singing in the morning] [V: was] [ARG2: a welcome sound] .']
Lstm
['[ARG0: The birds] [V: singing] [ARGM-TMP: in the morning] was a welcome sound .', '[ARG1: The birds singing in the morning] [V: was] [ARG2: a welcome sound] .']



The singing of birds in the morning was a welcome sound.
BERT
['[ARG1: The singing of birds in the morning] [V: was] [ARG2: a welcome sound] .']
Lstm
['[ARG1: The singing of birds in the morning] [V: was] [ARG2: a welcome sound] .']





In [ ]:
sentences=["It's running low ","It is running low"]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

It's running low 
BERT
["It [V: 's] running low", "[ARG0: It] 's [V: running] [ARGM-MNR: low]"]
Lstm
["It [V: 's] running low", "[ARG1: It] 's [V: running] [ARG2: low]"]



It is running low
BERT
['It [V: is] running low', '[ARG1: It] is [V: running] [ARGM-MNR: low]']
Lstm
['[ARG1: It] [V: is] [ARG2: running low]', '[ARG1: It] is [V: running] [ARG2: low]']





## RARITY

### Slang Verbs 

In [ ]:
predictor_Bert.predict("I really want to do it"),predictor_Bert.predict("I really wanna do it")
predictor_Bert.predict("I am gonna do it"),predictor_Bert.predict("I am going to do it")
[gimmie - give me, ]

({'verbs': [{'verb': 'want',
    'description': '[ARG0: I] [ARGM-ADV: really] [V: want] [ARG1: to do it]',
    'tags': ['B-ARG0', 'B-ARGM-ADV', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']},
   {'verb': 'do',
    'description': '[ARG0: I] really want to [V: do] [ARG1: it]',
    'tags': ['B-ARG0', 'O', 'O', 'O', 'B-V', 'B-ARG1']}],
  'words': ['I', 'really', 'want', 'to', 'do', 'it']},
 {'verbs': [{'verb': 'wanna',
    'description': '[ARG0: I] [ARGM-ADV: really] [V: wanna] [ARG1: do it]',
    'tags': ['B-ARG0', 'B-ARGM-ADV', 'B-V', 'B-ARG1', 'I-ARG1']},
   {'verb': 'do',
    'description': '[ARG0: I] really wanna [V: do] [ARG1: it]',
    'tags': ['B-ARG0', 'O', 'O', 'B-V', 'B-ARG1']}],
  'words': ['I', 'really', 'wanna', 'do', 'it']})

### Modern verbs

### old verbs

# AROUGMENTS CLASSIFICATION

In [9]:
#In this folder we have the data for the predicate classidication task. Change the path accordingly if files were moved.
path="./Data/Argument_classification/"
results_path=f"./results_{model}_Argument_Classification.json"
## we initialize also the dictionary that will contain the results
with open(results_path, 'w') as f:
    json.dump({"test":"failure_rate"},f)

In [10]:

def eval_predictor_INV(sentences,predictor,function):
  """
  This function evaluates the predictor by using the function to validate the predictions.
  function is the name a function that takes two predictions and returns a boolean
  """
  fails=0
 
  f = getattr(utils, function)
  for pair in sentences:
    
    s1,s2=pair

    pred1 = predictor.predict(s1)
    pred2 = predictor.predict(s2) 
    
    match=f(pred1,pred2,verbose=False)
    
    if not (match):
      fails+=1

  return (fails/len(sentences)*100)

## VOCABULAIRTY+POS

### Pronouns vs Entity

In [11]:
with open(path+"Pronouns_sents.json","r") as f:
    sentences=json.load(f)

sentences=list(sentences.items())

failure_rate=eval_predictor_INV(sentences,predictor,"validate_INV_allverbs_ARGset")

print(f"Rate of failure with : ",failure_rate)

append_to_results(results_path,{f"Pronouns_sents":failure_rate})


Rate of failure with :  37.5


## TAXONOMY


In [28]:
pred=predictor_Bert.predict("I killed a rabbit with a knife")
pred

{'verbs': [{'verb': 'killed',
   'description': '[ARG0: I] [V: killed] [ARG1: a rabbit] [ARG2: with a knife]',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARG2',
    'I-ARG2',
    'I-ARG2']}],
 'words': ['I', 'killed', 'a', 'rabbit', 'with', 'a', 'knife']}

In [40]:
pred=predictor_Bert.predict("Alice eats a chicken with a fork.")
pred

{'verbs': [{'verb': 'eats',
   'description': '[ARG0: Alice] [V: eats] [ARG1: a chicken] [ARGM-MNR: with a fork] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-MNR',
    'I-ARGM-MNR',
    'I-ARGM-MNR',
    'O']}],
 'words': ['Alice', 'eats', 'a', 'chicken', 'with', 'a', 'fork', '.']}

In [43]:
s=['Lukas eats a chicken with a feather.', 'Lukas eats a chicken with a stick.']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken with a feather] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'a', 'feather', '.']}
{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARGM-MNR: with a stick] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'I-ARGM-MNR', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'a', 'stick', '.']}


In [44]:
s=['Lukas eats a chicken with hands.', 'Lukas eats a chicken with olives.']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARGM-MNR: with hands] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'hands', '.']}
{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken with olives] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'olives', '.']}


In [46]:
s=['Lukas eats a chicken with tomatoes.', 'Lukas eats a chicken with forks.']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARG2: with tomatoes] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'tomatoes', '.']}
{'verbs': [{'verb': 'eats', 'description': '[ARG0: Lukas] [V: eats] [ARG1: a chicken] [ARGM-MNR: with forks] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'O']}], 'words': ['Lukas', 'eats', 'a', 'chicken', 'with', 'forks', '.']}


In [47]:
s=['I drink whisky with Ice', 'I drink wiskey with Jhon']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'drink', 'description': '[ARG0: I] [V: drink] [ARG1: whisky] [ARGM-MNR: with Ice]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['I', 'drink', 'whisky', 'with', 'Ice']}
{'verbs': [{'verb': 'drink', 'description': '[ARG0: I] [V: drink] [ARG1: wiskey] [ARGM-COM: with Jhon]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-COM', 'I-ARGM-COM']}], 'words': ['I', 'drink', 'wiskey', 'with', 'Jhon']}


In [55]:
s=['I have breakfast with Coffee', 'I have breakfast with Sunny']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'have', 'description': '[ARG0: I] [V: have] [ARG1: breakfast] [ARGM-MNR: with Coffee]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['I', 'have', 'breakfast', 'with', 'Coffee']}
{'verbs': [{'verb': 'have', 'description': '[ARG0: I] [V: have] [ARG1: breakfast] [ARGM-COM: with Sunny]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-COM', 'I-ARGM-COM']}], 'words': ['I', 'have', 'breakfast', 'with', 'Sunny']}


In [57]:
s=['I come by bike', 'I come by noon']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'come', 'description': '[ARG1: I] [V: come] [ARGM-MNR: by bike]', 'tags': ['B-ARG1', 'B-V', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['I', 'come', 'by', 'bike']}
{'verbs': [{'verb': 'come', 'description': '[ARG1: I] [V: come] [ARGM-TMP: by noon]', 'tags': ['B-ARG1', 'B-V', 'B-ARGM-TMP', 'I-ARGM-TMP']}], 'words': ['I', 'come', 'by', 'noon']}


In [59]:
s=['Lukas saw a man with a knife', 'Lukas saw a man with binoculars']
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'saw', 'description': '[ARG0: Lukas] [V: saw] [ARG1: a man with a knife]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1']}], 'words': ['Lukas', 'saw', 'a', 'man', 'with', 'a', 'knife']}
{'verbs': [{'verb': 'saw', 'description': '[ARG0: Lukas] [V: saw] [ARG1: a man] [ARGM-MNR: with binoculars]', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['Lukas', 'saw', 'a', 'man', 'with', 'binoculars']}


In [63]:
s=["Lukas having a beer with a friend.","Lukas having a beer with a lemon."]
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'having', 'description': '[ARG0: Lukas] [V: having] [ARG1: a beer] [ARGM-COM: with a friend] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-COM', 'I-ARGM-COM', 'I-ARGM-COM', 'O']}], 'words': ['Lukas', 'having', 'a', 'beer', 'with', 'a', 'friend', '.']}
{'verbs': [{'verb': 'having', 'description': '[ARG0: Lukas] [V: having] [ARG1: a beer] [ARG2: with a lemon] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['Lukas', 'having', 'a', 'beer', 'with', 'a', 'lemon', '.']}


In [64]:
s=["Lukas helps a man with a disability.","Lukas helps a man with a task."]
for x in s:
    pred=predictor_Bert.predict(x)
    print(pred)

{'verbs': [{'verb': 'helps', 'description': '[ARG0: Lukas] [V: helps] [ARG2: a man with a disability] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['Lukas', 'helps', 'a', 'man', 'with', 'a', 'disability', '.']}
{'verbs': [{'verb': 'helps', 'description': '[ARG0: Lukas] [V: helps] [ARG2: a man] [ARG1: with a task] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O']}], 'words': ['Lukas', 'helps', 'a', 'man', 'with', 'a', 'task', '.']}


## NER

### Very Long Entities

## ROBUSTNESS

### Typos

In [ ]:
for i in range(1,5):#becuase we have 4 files like that

    with open(path+f"sents_{i}_typos.json", 'r') as f:
        sentences = json.load(f)

    sentences=list(sentences.items())

    rate_typos_n=eval_predictor_INV(sentences,predictor,"evaluate_INV_alltags")

    print(f"Rate of failure with {i} typos per sentence: ",rate_typos_n)

    append_to_results(results_path,{f"sents_{i}_typos":rate_typos_n})


Rate of failure with 1 typos per sentence:  28.999999999999996
Rate of failure with 2 typos per sentence:  49.0
Rate of failure with 3 typos per sentence:  65.0
Rate of failure with 4 typos per sentence:  76.0


## PARAPHRASING

### word orders 

In [ ]:
sentences=["the coach teaches to the team coordination", "The coach teaches coordination to the team"]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

the coach teaches to the team coordination
['[ARG0: the coach] [V: teaches] [ARG2: to the team coordination]']



The coach teaches coordination to the team
['[ARG0: The coach] [V: teaches] [ARG1: coordination to the team]']





In [ ]:
sentences=["I like that mark has not beeing able to reduce sigarets despite his mother dying of lung cancer"]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I like that mark has not beeing able to reduce sigarets despite his mother dying of lung cancer
['[ARG0: I] [V: like] [ARG1: that mark has not beeing able to reduce sigarets despite his mother dying of lung cancer]', 'I like that mark [V: has] not beeing able to reduce sigarets despite his mother dying of lung cancer', 'I like that [ARG1: mark] has [ARGM-NEG: not] [V: beeing] [ARGM-PRD: able to reduce sigarets] [ARGM-ADV: despite his mother dying of lung cancer]', 'I like that [ARG0: mark] has not beeing able to [V: reduce] [ARG1: sigarets] despite his mother dying of lung cancer', 'I like that mark has not beeing able to reduce sigarets despite [ARG1: his mother] [V: dying] [ARGM-CAU: of lung cancer]']





In [ ]:
sentences=["I give to you kisses", "I give kisses to you"]
for s in sentences:
  print(s)
  pred=predictor_bilstm.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I give to you kisses
['[ARG0: I] [V: give] [ARG2: to you kisses]']



I give kisses to you
['[ARG0: I] [V: give] [ARG1: kisses] [ARG2: to you]']





In [ ]:
sentences=["you are stupid"]
for s in sentences:
  print(s)
  
  pred=predictor_lstm.predict(s)
  print(pred)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

you are stupid


NameError: ignored

In [ ]:
sentences=[" looking eating kisses Robert at lunchtime in the park"]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

 looking eating kisses Robert at lunchtime in the park
BERT
['[V: looking] [ARG1: eating kisses Robert at lunchtime in the park]', 'looking [V: eating] [ARG1: kisses Robert] [ARGM-TMP: at lunchtime] [ARGM-LOC: in] [ARGM-TMP: the park]']
Lstm
['[V: looking] [ARG1: eating kisses Robert at lunchtime in the park]', 'looking [V: eating] [ARG1: kisses Robert at lunchtime] [ARGM-LOC: in the park]']





In [ ]:
sentences=["I give to you love", "I give love to you"]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I give to you love
['[ARG0: I] [V: give] [ARG2: to you love]', 'I give to [ARG0: you] [V: love]']



I give love to you
['[ARG0: I] [V: give] [ARG1: love] [ARG2: to you]']





### passive and actrive trasnformarion

In [15]:
with open(path+"activepassive_sentences.json", 'r') as f:
    sentences = json.load(f)

sentences=list(sentences.items())

rate=eval_predictor_INV(sentences,predictor,"evaluate_INV_sameArgs")

print(f"Rate of failure with for INV activepassive ",rate)

append_to_results(results_path,{f"activepassive_sents":rate})


Rate of failure with for INV activepassive  0.0


### Questions phrasing
Idea for testing is to take all the arguments from from the verb that is the same(not the auxialiry).
 All the arguments should be the same except for the verb.

In [11]:
a=[["She studied hard for the exam.", "Did she study hard for the exam?"],
["He is working from home today.", "Is he working from home today?"],
['You give the book to me.', 'Do you give the book to me?'],
['They are going to the cinema.', 'Are they going to the cinema?'],
['the dog is barking', 'is the dog barking?'],
['We love pizza with spicy sauce.', 'Do we love pizza with spicy sauce?'],
['Pelicans are flying over the lake.', 'Are pelicans flying over the lake?'],
['The cat is sleeping on the sofa.', 'Is the cat sleeping on the sofa?'],
['The dog is barking at the cat.', 'Is the dog barking at the cat?'],
['In Moscow, it is snowing.', 'Is it snowing in Moscow?'],
['There are a bunch of keys on the table.', 'Are there a bunch of keys on the table?'],
['Some people like to eat insects.', 'Do some people like to eat insects?'],
['I would feel bad if I were you.', 'Would I feel bad if I were you?'],
['You have been to Paris before', 'Have you been to Paris before?']]


In [17]:
with open(path+"question_rephrase.json", 'r') as f:
    sentences = json.load(f)

sentences=list(sentences.items())

rate=eval_predictor_INV(sentences,predictor,"evaluate_INV_sameArgs")

print(f"Rate of failure with for INV in question rephrasing task ",rate)

append_to_results(results_path,{f"question_rephrase":rate})

Rate of failure with for INV in question rephrasing task  0.0


In [13]:
#sentences=list(sentences.items())

rate=eval_predictor_INV(a,predictor,"evaluate_INV_activepassive")

print(f"Rate of failure with for INV activepassive ",rate)

append_to_results(results_path,{f"activepassive_sents":rate})


different arguments


NameError: name 'exit' is not defined

## Coreference STUDYING


In [ ]:

pred1=predictor_Bert.predict("It annoys me the results of the election")
pred1


pred=predictor_Bert.predict("It Is great the results of the election")
pred

pred=predictor_Bert.predict("He was the one that did not want to go to the party")
for v in pred['verbs']:
  print(v['description'])
  

pred=predictor_Bert.predict("she is the sister that likes milk")
for v in pred['verbs']:
  print(v['description'])
  

relative_pronouns = {
    "who": "The person who stole my bike was caught by the police.",
    "whom": "The woman whom I saw at the store was carrying a lot of groceries.",
    "which": "The car which I rented for my vacation was very comfortable.",
    "whose": "The man whose wallet was lost reported the incident to the authorities.",
    "where": "The park where we had our picnic was very peaceful.",
    "when": "The time when we first met is a very special memory for me.",
    "that": "The book that I'm reading right now is very suspenseful.",
    "whoever": "Whoever finishes first gets a prize.",
    "whichever": "You can choose whichever movie you want to see.",
}
for x in relative_pronouns:
  print(x)
  pred=predictor_Bert.predict(relative_pronouns[x])
  for v in pred['verbs']:
    print(v['description'])
  
pred=predictor_Bert.predict("The partner of the lady, who is very old, has exacly my t-shirt")
pred
pred['verbs'][0]['description']

pred=predictor_Bert.predict("The partner of the lady who is very old, has exacly my t-shirt")
pred
pred['verbs'][0]['description']

pred=predictor_Bert.predict("The partner of the lady who is very old has exacly my t-shirt")
pred
pred['verbs'][0]['description']
pred=predictor_Bert.predict("The cars in the shop which is expensive, are exacly what i dream")
pred
pred['verbs'][0]['description']

Ok so the idea is the coreference u dont need here becuase SRL is sentence based very much and for every predicate
you have a sentence. So the coreference is not needed. But sometimes you have that in the same sentence.

In [204]:

pred1=predictor_Bert.predict("It is the key of Louis and the car is also his")
pred1

{'verbs': [{'verb': 'is',
   'description': '[ARG1: It] [V: is] [ARG2: the key of Louis] and the car is also his',
   'tags': ['B-ARG1',
    'B-V',
    'B-ARG2',
    'I-ARG2',
    'I-ARG2',
    'I-ARG2',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'is',
   'description': 'It is the key of Louis and [ARG1: the car] [V: is] [ARGM-ADV: also] [ARG2: his]',
   'tags': ['O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B-ARG1',
    'I-ARG1',
    'B-V',
    'B-ARGM-ADV',
    'B-ARG2']}],
 'words': ['It',
  'is',
  'the',
  'key',
  'of',
  'Louis',
  'and',
  'the',
  'car',
  'is',
  'also',
  'his']}

## AMBIGUITY

#### PP -ambiguity
 

In [ ]:
sentences="I went to the resturant by the Hutsin", "I went to the resturant by"
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I went to the resturant by the Hutsin
['[ARG0: I] [V: went] [ARG4: to the resturant by the Hutsin]']



I went to the resturant by bike
['[ARG0: I] [V: went] [ARG4: to the resturant] [ARGM-MNR: by bike]']





In [ ]:
["I fixed the car with a red logo","I fixed the car with a wretch"]
["I bought a computer with GPU"," I bought a computer wit bitcoins"]


# PROBABLY NOT DOING THEM any mre





### Overlapping spans


### INCISI

In [194]:
sentences=["This, friend, is epic ","This friend is epic"]
for s in sentences:
  print(s)
  pred=predictor_Bert.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

This, friend, is epic 
['[ARG1: This ,] [ARGM-DIS: friend] , [V: is] [ARG2: epic]']



This friend is epic
['[ARG1: This friend] [V: is] [ARG2: epic]']





In [ ]:
(I might stay, tonight, with you)

### DIfferent frames for labels recognition
With mutiple verbs, argoument confusion or shiet

### Saxon genitive

In [ ]:
paired_sentences = [    ["The president's decision to withdraw troops from the region caused controversy among military leaders.",      "The decision of the president to withdraw troops from the region caused controversy among military leaders."],
    ["The CEO's success in turning around the struggling company was due in large part to her innovative marketing strategies.",      "The success of the CEO in turning around the struggling company was due in large part to her innovative marketing strategies."],
    ["The director's vision for the film was not fully realized due to budget constraints and scheduling conflicts.",      "The vision of the director for the film was not fully realized due to budget constraints and scheduling conflicts."],
    ["The professor's lecture on the history of ancient Rome was attended by a packed auditorium of eager students.",      "The lecture on the history of ancient Rome of the professor was attended by a packed auditorium of eager students."],
    ["The artist's latest work, a sculpture made entirely of recycled materials, was featured in a prominent gallery exhibition.",      "The latest work of the artist, a sculpture made entirely of recycled materials, was featured in a prominent gallery exhibition."],
    ["The athlete's rigorous training regimen and strict diet led to her record-breaking performance at the championship.",      "The rigorous training regimen and strict diet of the athlete led to her record-breaking performance at the championship."],
    ["The author's use of symbolism and metaphor in her novel added depth and complexity to the story.",      "The use of symbolism and metaphor in her novel of the author added depth and complexity to the story."]
]


### Intransitive verbs

In [ ]:
sentences = ['The sun rises in the east.',
             'He slept for 10 hours.',
             'They danced all night.',
             'The flowers bloomed in the spring.',
             'The baby cried all night.',
             'The cat meowed loudly.',
             'The car stopped suddenly.',
             'She ran around the park.',
             'The plane took off at 9 am.',
             'The wind blew fiercely.',
             'The tree swayed in the wind.',
             'The river flows downstream.',
             'The coffee brewed in the pot.',
             'The music played softly in the background.',
             'The bird chirped early in the morning.',
             'The children laughed and played.',
             'The fire crackled in the fireplace.',
             'The leaves fell from the trees.',
             'The boat sailed across the ocean.',
             'The train arrived at the station on time.']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

The sun rises in the east.
['[ARG1: The sun] [V: rises] [ARGM-LOC: in the east] .']



He slept for 10 hours.
['[ARG0: He] [V: slept] [ARGM-TMP: for 10 hours] .']



They danced all night.
['[ARG0: They] [V: danced] [ARGM-TMP: all night] .']



The flowers bloomed in the spring.
['[ARG0: The flowers] [V: bloomed] [ARGM-TMP: in the spring] .']



The baby cried all night.
['[ARG0: The baby] [V: cried] [ARGM-TMP: all night] .']



The cat meowed loudly.
['[ARG0: The cat] [V: meowed] [ARGM-MNR: loudly] .']



The car stopped suddenly.
['[ARG1: The car] [V: stopped] [ARGM-MNR: suddenly] .']



She ran around the park.
['[ARG0: She] [V: ran] [ARGM-DIR: around the park] .']



The plane took off at 9 am.
['[ARG1: The plane] [V: took] off [ARGM-TMP: at 9 am] .']



The wind blew fiercely.
['[ARG1: The wind] [V: blew] [ARGM-MNR: fiercely] .']



The tree swayed in the wind.
['[ARG1: The tree] [V: swayed] [ARGM-LOC: in the wind] .']



The river flows downstream.
['[ARG1: The river] [V: flows] 

In [ ]:
sentences = ['The snow fell gently from the sky.',
             'She laughed uncontrollably at the joke.',
             'The sun set over the horizon.',
             'The children played happily in the park.',
             'The fire burned brightly in the fireplace.',
             'The wind howled outside the window.',
             'The cake baked for an hour.',
             'The door opened slowly.',
             'The crowd cheered loudly at the concert.',
             'The athlete ran around the track.',
             'The tree grew tall and strong.',
             'The river flowed peacefully.',
             'The bird flew gracefully through the air.',
             'The fish swam quickly in the water.',
             'The dog barked loudly at the mailman.',
             'The car drove smoothly down the road.',
             'The plane flew high above the clouds.',
             'The moon shone brightly in the sky.',
             'The stars twinkled in the night sky.',
             'The flowers swayed in the breeze.']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

The snow fell gently from the sky.
['[ARG1: The snow] [V: fell] [ARGM-MNR: gently] [ARG3: from the sky] .']



She laughed uncontrollably at the joke.
['[ARG0: She] [V: laughed] [ARGM-MNR: uncontrollably] [ARG2: at the joke] .']



The sun set over the horizon.
['[ARG1: The sun] [V: set] [ARG2: over the horizon] .']



The children played happily in the park.
['[ARG0: The children] [V: played] [ARGM-MNR: happily] [ARGM-LOC: in the park] .']



The fire burned brightly in the fireplace.
['[ARG1: The fire] [V: burned] [ARGM-MNR: brightly] [ARGM-LOC: in the fireplace] .']



The wind howled outside the window.
['[ARG0: The wind] [V: howled] [ARGM-LOC: outside the window] .']



The cake baked for an hour.
['[ARG1: The cake] [V: baked] [ARGM-TMP: for an hour] .']



The door opened slowly.
['[ARG1: The door] [V: opened] [ARGM-MNR: slowly] .']



The crowd cheered loudly at the concert.
['[ARG0: The crowd] [V: cheered] [ARGM-MNR: loudly] [ARG1: at the concert] .']



The athlete ran around 

In [ ]:
sent="Many people keep falling for this CON - GAME that lower taxes on the rich benefits everyone . "
predictor.predict(sent)

{'verbs': [{'verb': 'keep',
   'description': '[ARG0: Many people] [V: keep] [ARG1: falling for this CON - GAME that lower taxes on the rich benefits everyone] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'falling',
   'description': '[ARG1: Many people] keep [V: falling] [ARGM-PRP: for this CON - GAME that lower taxes on the rich benefits everyone] .',
   'tags': ['B-ARG1',
    'I-ARG1',
    'O',
    'B-V',
    'B-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'O']}],
 'words': ['Many',
  'people',
  'keep',
  'falling',
  'for',
  'this',
  'CON',
  '-',
  'GAME',
  'that',
  'lower'

In [ ]:
predictor.predict("He walked the dog to the park and back.")

{'verbs': [{'verb': 'walked',
   'description': '[ARG0: He] [V: walked] [ARG1: the dog] [ARGM-GOL: to the park and back] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'O']}],
 'words': ['He',
  'walked',
  'the',
  'dog',
  'to',
  'the',
  'park',
  'and',
  'back',
  '.']}

In [ ]:
predictor.predict("He walked with the dog to the park and back.")

{'verbs': [{'verb': 'walked',
   'description': '[ARG0: He] [V: walked] [ARGM-COM: with the dog] [ARGM-GOL: to the park and back] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARGM-COM',
    'I-ARGM-COM',
    'I-ARGM-COM',
    'B-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'O']}],
 'words': ['He',
  'walked',
  'with',
  'the',
  'dog',
  'to',
  'the',
  'park',
  'and',
  'back',
  '.']}

In [ ]:
predictor.predict("I see a man with a telescope under his arm")

{'verbs': [{'verb': 'see',
   'description': '[ARG0: I] [V: see] [ARG1: a man with a telescope under his arm]',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1']}],
 'words': ['I',
  'see',
  'a',
  'man',
  'with',
  'a',
  'telescope',
  'under',
  'his',
  'arm']}

In [ ]:
predictor.predict("The plane took a large turn")

{'verbs': [{'verb': 'took',
   'description': '[ARG0: The plane] [V: took] [ARG1: a large turn]',
   'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']}],
 'words': ['The', 'plane', 'took', 'a', 'large', 'turn']}

### Parenthetical Elements 